## Consulta de datos sin servidores ni bases de datos mediante Amazon S3 Select

Anteriormente, era necesario cargar los datos en una base de datos para poder consultarlos. Además de implementar una base de datos, el cliente habría necesitado implementar una aplicación y un sitio web para habilitar la búsqueda. En lugar de desplegar una base de datos y los recursos asociados, aprovechamos una característica de S3 llamada [S3 Select](https://docs.aws.amazon.com/AmazonS3/latest/userguide/selecting-content-from-objects.html) para crear una herramienta de búsqueda en la guía telefónica completamente sin servidor.

Los clientes aprovechan Amazon S3 para almacenar y proteger cualquier cantidad de datos sin aprovisionar almacenamiento ni administrar infraestructura. Amazon S3 Select y Amazon S3 Glacier Select permiten a los clientes ejecutar consultas SQL en lenguaje de consulta estructurado directamente en los datos almacenados en S3 y Amazon S3 Glacier. Con S3 Select, solo tiene que almacenar sus datos en S3 y realizar consultas mediante instrucciones SQL para filtrar el contenido de los objetos de S3 y recuperar únicamente los datos que necesita. Al recuperar solo un subconjunto de los datos, los clientes reducen la cantidad de datos que Amazon S3 transfiere, lo que reduce el coste y la latencia de la recuperación de estos datos. La reducción del costo y la complejidad permite a los clientes de AWS avanzar más rápido y reducir la cantidad de tiempo necesaria para ofrecer valor a sus empresas y a sus clientes.

S3 Select funciona con objetos almacenados en formato CSV, JSON o Apache Parquet. S3 Select también admite compresión en objetos CSV y JSON con GZIP o BZIP2, y objetos cifrados en el lado del servidor. Puede realizar consultas SQL mediante los SDK de AWS, la API REST de SELECT Object Content, la interfaz de línea de comandos de AWS (CLI de AWS) o la consola de administración de AWS.

In [22]:
import boto3
import json
import math

session = boto3.Session(profile_name='workshop')
s3 = session.client('s3')

In [16]:
# Function to convert bytes to KB, MB, etc
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [23]:
#Upload the file to S3
bucket_name = 'doc-example-001-sensitive-data'
file_name = 'compagnie_français.csv'

s3.upload_file(file_name, bucket_name, file_name)

In [24]:
resp = s3.select_object_content(
    Bucket=bucket_name,
    Key=file_name,
    ExpressionType='SQL',
    Expression="SELECT * FROM s3object s where s.\"Names\" = 'John Doe'",
    InputSerialization = {'CSV': {"FileHeaderInfo": "Use"}, 'CompressionType': 'NONE'},
    OutputSerialization = {'CSV': {}},
)

In [25]:
for event in resp['Payload']:
    if 'Records' in event:
        records = event['Records']['Payload'].decode('utf-8')
        print(records)
    elif 'Stats' in event:
        statsDetails = event['Stats']['Details']
        print("Stats details bytesScanned: ", convert_size(statsDetails['BytesScanned']))
        print("Stats details bytesProcessed: ", convert_size(statsDetails['BytesProcessed']))
        print("Stats details bytesReturned: ", convert_size(statsDetails['BytesReturned']))

43DC7847-426D-374B-66DB-F4235668650B,John Doe,johndoe@example.com,Example Corp.,29-Jan-20,351 322 706-00000
BFEB70BE-49D9-91FA-AFAC-FBF90F071280,John Doe,johndoe@example.net,Example Corp.,22-Feb-21,863811113-00001
44F79BF2-2923-1D16-30E9-3B24EB960945,John Doe,johndoe@example.org,Example Corp.,11-Dec-20,898606835-00004
19F22A2E-E477-B93A-BDE1-60A64089717D,John Doe,johndoe@example.io,Example Corp.,15-Dec-20,487726333-00007
5DFD2087-EA86-B01B-7514-55846AEFE126,John Doe,johndoe@example.edu,Example Corp.,28-Jul-19,757331780-00002
63AB89C4-3AE7-0E4D-059D-30B722ECE5B7,John Doe,johndoe@example.co,Example Corp.,27-Sep-19,607991817-00000
46894AC0-91FC-A832-67F7-C7EF743F1952,John Doe,johndoe@example.ca,Example Corp.,9-Jun-19,797558954-00005

Stats details bytesScanned:  21.94 KB
Stats details bytesProcessed:  21.94 KB
Stats details bytesReturned:  747.0 B
